In [1]:
BASE_PATH = '/content/drive/MyDrive/Health/'
DATASET_PATH = BASE_PATH + 'dataset/'
VOCAB_PATH = '/content/drive/MyDrive/Health/vocab.txt'

TEST_SET_PATH = BASE_PATH + 'test_1000-1099.fas'

URL_DRIVE = 'https://drive.google.com/uc?id='

# BASIC DATASET
# G_TRAIN_FILE = URL_DRIVE + '1-1eXBY8yHHmWDlzr-gZXAX8POor-P1xs'
# G_DEV_FILE   = URL_DRIVE + '1bo_JKi6SbcRH8l-2GhIcdl1AnWvNaOYK'
# G_TEST_FILE  = URL_DRIVE + '1-2zqIzM3FgsfTi4jJmC2wBb9KHjKk_xx'

# K512 DATASET
G_TRAIN_FILE = URL_DRIVE + '1-ESFbZab0N7npe1Q01549CqEKLfQoCDd'
G_VALID_FILE   = URL_DRIVE + '1-GSyPyObFCwFqa8amuizDIQbE51sPAKw'
G_TEST_FILE  = URL_DRIVE + '1-8EQbga_UpaV3wwBxuUc7dzij_LJzQ3a'

# K67100 DATASET
# G_TRAIN_FILE = URL_DRIVE + '1-1slsqddGwmcKNkj2b67Sg7xJvPjh5zI'
# G_VALID_FILE   = URL_DRIVE + '1-DPawqObrh3rRCtl8gjlJJhyUm99k4mU'
# G_TEST_FILE  = URL_DRIVE + '1-EV62T1oc5WSWjCBh1poGM2-2EGzgUnP'

TRAIN_FILE = 'dataset/train/train.tsv'
VALID_FILE = 'dataset/train/dev.tsv'
TEST_FILE = 'dataset/test/dev.tsv'

# ID_LABELS_K512
TRAIN_ID_LABELS = URL_DRIVE + '1_D-b0-R4ybQUqzjA8rHxjvNQxBF36V-G'
VALID_ID_LABELS = URL_DRIVE + '1-P13Uomv9SeBQondNdqNIKsYgWVkpbrj'
TEST_ID_LABELS = URL_DRIVE + '1SRO5FxufHQcjGHFTXHuXy-M_xDAkqUWm'

# ID_LABELS_K67100
# TRAIN_ID_LABELS = URL_DRIVE + '1-LpjEfcq2mx9lX2H_5eCnEn-oVhGX3Ou'
# VALID_ID_LABELS = URL_DRIVE + '1-O_aT-5332WSFqM2RhF9td1w__G6Zvk6'
# TEST_ID_LABELS = URL_DRIVE + '1-NAPO59OR1Fv6dtpmxw1N6MBTlwLe6YF'

TRAIN_ID_LABELS_FILE = 'dataset/train/train_id_labels.txt'
VALID_ID_LABELS_FILE = 'dataset/train/dev_id_labels.txt'
TEST_ID_LABELS_FILE = 'dataset/test/dev_id_labels.txt'

# EMBEDDINGS K512
EMB_TRAIN = URL_DRIVE + '1M6UwhFW9UZuCMAcz-vhtuABekM2f7o1m'
EMB_VALID = URL_DRIVE + '1k6bDa1iIWVLfROKvuhvUBrrwhF4KBCxN'
EMB_TEST = URL_DRIVE + '1XDjg9IStdPRHyh7wuVwHiN_mMwQdILeI'

EMB_TRAIN_FILE = 'embeddings/train/'
EMB_VALID_FILE = 'embeddings/valid/'
EMB_TEST_FILE = 'embeddings/test/'

K = 6
SPLIT_SIZE = 3584 #512 sequences

## Install

In [ ]:
import os.path
from IPython.core.display import HTML

RESTART_FILE = 'restart.txt'

if not os.path.exists(RESTART_FILE):
    !pip -q install gdown
    !pip -q install Bio

    !git clone https://github.com/jerryji1993/DNABERT
    %cd DNABERT
    !pip install --editable .
    %cd examples
    !pip install -r requirements.txt
    !touch $RESTART_FILE
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Imports

In [2]:
from Bio import SeqIO
from itertools import product
import random
import glob
import os
import time
import shutil

import gdown

import torch
from transformers import BertModel, BertConfig, DNATokenizer

from sklearn.decomposition import PCA

## Dataset

In [10]:
os.makedirs('dataset', exist_ok=True)
os.makedirs('dataset/train', exist_ok=True)
os.makedirs('dataset/test', exist_ok=True)
os.makedirs('output', exist_ok=True)

In [ ]:
# downloading dataset seq-id
gdown.download(G_TRAIN_FILE, TRAIN_FILE, quiet=False)
gdown.download(G_VALID_FILE, VALID_FILE, quiet=False)
gdown.download(G_TEST_FILE, TEST_FILE, quiet=False)

# downloading dataset id-label
gdown.download(TRAIN_ID_LABELS, TRAIN_ID_LABELS_FILE, quiet=False)
gdown.download(VALID_ID_LABELS, VALID_ID_LABELS_FILE, quiet=False)
gdown.download(TEST_ID_LABELS, TEST_ID_LABELS_FILE, quiet=False)

In [ ]:
!ls dataset/train

In [ ]:
# !rm -r dataset/*
# !rm -r test/*

# !rm -r output/*
# !rm -r prediction/*

In [ ]:
# filenames = ['train.tsv', 'dev.tsv', 'test.tsv']

# def splitSeq2Lines(seq, split_size):
#     with open('temp/' + filename, 'r') as f:
#         with open('dataset/' + filename, 'w') as out:
#             print(filename)
            
#             line = f.readline()
#             out.write(line) # writing header
#             ctr = 0
#             while line:
#                 # do stuff with line
#                 line = f.readline()
#                 start_pos = 0
#                 end_pos = split_size
#                 line = line.split('\t')
#                 while(start_pos<=len(line[0])):
#                     out.write(line[0][start_pos:end_pos] + '\t' + str(ctr) + '\n')
#                     start_pos+=split_size
#                     end_pos+=split_size
#                 ctr += 1
    

# def splitSequence(filename, split_size=SPLIT_SIZE):
#     with open('temp/' + filename, 'r') as f:
#         with open('dataset/' + filename, 'w') as out:
#             print(filename)
#             lines = f.readlines()
#             out.write(lines[0])
#             lines = lines[1:]
#             random.shuffle(lines)
#             ctr = 0
#             for line in lines:
#                 if ctr != len(lines) - 1:
#                     line = line.split('\t')
#                     out.write(line[0][0:split_size] + '\t' + line[1])
#                 else:
#                     line = line.split('\t')
#                     out.write(line[0][0:split_size] + '\t' + line[1].split('\n')[0])
#                 ctr += 1

# # SPLIT_SIZE = 671
# print('Split Size: ' + str(SPLIT_SIZE))
# for filename in filenames:
#     splitSeq2Lines(filename, SPLIT_SIZE)

## Finetuning

In [ ]:
# using old dataset uploaded in kaggle/meningitis
#!cp -r ../input/menigitis/dataset_67100/ ./
# removing old runs of the model
#!rm dataset_67100/train/cached_*

In [ ]:
!ls dataset/train
!ls dataset/test

In [ ]:
from IPython.display import FileLink

FileLink(r'dataset_67100_shuffled/train/dev.tsv')

In [ ]:
# #trying to shuffle the dev set
# import random

# os.makedirs('dataset_67100_shuffled', exist_ok=True)
# os.makedirs('dataset_67100_shuffled/train', exist_ok=True)
# os.makedirs('dataset_67100_shuffled/test', exist_ok=True)

# with open('dataset_67100/train/train.tsv', 'r') as input:
#     with open('dataset_67100_shuffled/train/train.tsv', 'w') as output:
#         line = input.readline()
#         output.write(line)
#         data = []
#         while line:
#             line = input.readline()
#             data.append(line)
#         random.shuffle(data)
#         print('Shuffled ' + str(len(data)) + ' sequences')
#         for l in data:
#             output.write(l)

In [ ]:
KMER = K
MODEL_PATH='../input/menigitis/DNABERT6'
DATA_PATH='dataset/train/'
#DATA_PATH = 'dataset_67100/train/'
OUTPUT_PATH='output'

EPOCHS = 2.0
MAX_SEQ_LENGTH = 256
BATCH_SIZE = 1

In [ ]:
with torch.no_grad():
    !python DNABERT/examples/run_finetune.py \
        --model_type dnalong \
        --tokenizer_name=dna$KMER \
        --model_name_or_path $MODEL_PATH \
        --task_name dnaprom \
        --do_train \
        --do_eval \
        --data_dir $DATA_PATH \
        --max_seq_length $MAX_SEQ_LENGTH \
        --per_gpu_eval_batch_size=$BATCH_SIZE   \
        --per_gpu_train_batch_size=$BATCH_SIZE   \
        --learning_rate 2e-4 \
        --num_train_epochs $EPOCHS \
        --output_dir $OUTPUT_PATH \
        --evaluate_during_training \
        --logging_steps 100 \
        --save_steps 4000 \
        --warmup_percent 0.1 \
        --hidden_dropout_prob 0.1 \
        --overwrite_output \
        --weight_decay 0.01 \
        --n_process 12

In [ ]:
!ls output

In [ ]:
!more output/eval_results.txt

In [ ]:
# 05/03/2022 11:28:33 - INFO - __main__ -   ***** Eval results  *****
# 05/03/2022 11:28:33 - INFO - __main__ -     acc = 0.9833333333333333
# 05/03/2022 11:28:33 - INFO - __main__ -     auc = 0.9916666666666666
# 05/03/2022 11:28:33 - INFO - __main__ -     f1 = 0.983328702417338
# 05/03/2022 11:28:33 - INFO - __main__ -     mcc = 0.9672041516493516
# 05/03/2022 11:28:33 - INFO - __main__ -     precision = 0.9838709677419355
# 05/03/2022 11:28:33 - INFO - __main__ -     recall = 0.9833333333333334

In [ ]:
# !more output/eval_results.txt

In [ ]:
probs = np.loadtxt("output/eval_results.txt")
print(probs.shape)
probs

In [ ]:
# import numpy as np
# val_output = 'output/eval_results.txt'
# # probs = np.load('prediction/pred_results.npy')
# probs = np.loadtxt(val_output)
# # preds = np.argmax(probs)
# preds = []
# for prob in probs:
#     if prob > 0.5:
#         preds.append(1)
#     else:
#         preds.append(0)
# print(len(preds))
# print(preds)

## Test

In [ ]:
KMER = K
MODEL_PATH='output'
# DATA_PATH='test'
DATA_PATH = 'dataset_67100/test'
PREDICTION_PATH='prediction'

In [ ]:
!python DNABERT/examples/run_finetune.py \
    --model_type dna \
    --tokenizer_name=dna$KMER \
    --model_name_or_path $MODEL_PATH \
    --task_name dnaprom \
    --do_predict \
    --data_dir $DATA_PATH  \
    --max_seq_length 512 \
    --per_gpu_pred_batch_size=6   \
    --output_dir $MODEL_PATH \
    --predict_dir $PREDICTION_PATH \
    --n_process 12

In [ ]:
!ls prediction

In [ ]:
probs = np.load('prediction/pred_results.npy')
print(probs.shape)
print(probs)

In [ ]:
import numpy as np
probs = np.load('prediction/pred_results.npy')
#probs = np.loadtxt(val_output)
# preds = np.argmax(probs)
preds = []
for prob in probs:
    if prob > 0.5:
        preds.append(1)
    else:
        preds.append(0)
print(len(preds))
print(preds)

In [ ]:
!ls dataset_67100/test/dev.tsv

In [ ]:
import pandas as pd
test_file = pd.read_csv('dataset_67100/test/dev.tsv', delimiter='\t')
test_file['label'].value_counts()

## Remove cache

In [ ]:
# !rm -r test/cached*
# !rm -r dataset/cached*

## Encodings

In [3]:
MODEL_PATH = '../input/menigitis/DNABERT6'
config = BertConfig.from_pretrained('https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/config.json')
tokenizer = DNATokenizer.from_pretrained('dna6')
    
def loadModel(model_path):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    model = BertModel.from_pretrained(model_path, config=config).to(device)
    return model

def getEmbeddings(model, sequence):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model_input = tokenizer.encode_plus(sequence, add_special_tokens=True, max_length=256)["input_ids"]
    model_input = torch.tensor(model_input, dtype=torch.long).to(device)
    model_input = model_input.unsqueeze(0)   # to generate a fake batch with batch size one

    output = model(model_input)
    return output[1][0].detach()

def get_number(number):
    return format(number, '04d')

def count_lines(filename):
    f = open(filename)                  
    lines = 0
    buf_size = 1024 * 1024
    read_f = f.read # loop optimization

    buf = read_f(buf_size)
    while buf:
        lines += buf.count('\n')
        buf = read_f(buf_size)

    return lines

def limit_decimals(number, decimals=2):
    return format(number, '.'+str(decimals)+'f')

def writeEmbeddings(input_file, output_path='embeddings', start_id=0, limit=100):
    start_time = time.time()
    if 'train.tsv' in input_file: f_type = 'train'
    elif '/train/dev.tsv' in input_file: f_type = 'valid'
    else: f_type = 'test'
    output_path = output_path + '/' + f_type + '/emb_seq'
    n_lines = count_lines(input_file)
    print('[' + input_file.split('/')[-2] + '] N_LINES: ' + str(n_lines))
    
    with open(input_file, 'r') as f_in:
        ctr = 0
        line = f_in.readline()
        current_id = start_id
        n_inserts = 0
        temp_dict={}
        while line:
            line = f_in.readline()
            splitted_line = line.split('\t')
            if len(splitted_line) < 2:
                torch.save(temp_dict, output_path + get_number(current_id) + '.pt')
                break
            seq = splitted_line[0]
            seq_id = int(splitted_line[1])
            if seq_id < start_id:
                ctr += 1
                continue
            emb = getEmbeddings(model, seq)
            if current_id == seq_id:
                # inserisci nel dizionario
                temp_dict[n_inserts] = emb
                n_inserts += 1
            else:
                # scrivi su file il dizionario
                torch.save(temp_dict, output_path + get_number(current_id) + '.pt')
                # aumenta current_id <- seq_id
                current_id = seq_id
                if current_id - start_id >= limit: # ==
                    break
                # svuota dizionario e inserisci la sequenza
                n_inserts = 0
                temp_dict = {}
                temp_dict[n_inserts] = emb
                

            ctr += 1
            if ctr % 100 == 0:
                percentage = limit_decimals(ctr/n_lines*100)
                elapsed_time = limit_decimals(time.time() - start_time)
                print(percentage + "% (" + str(current_id) + ") --- " + elapsed_time + " seconds ---")
#             if (current_id - start_id) % 100 == 0:
                
        print(ctr)
    

model = loadModel(MODEL_PATH)

In [ ]:
!rm -r embeddings/*

os.makedirs('embeddings', exist_ok=True)
os.makedirs('embeddings/train', exist_ok=True)
os.makedirs('embeddings/valid', exist_ok=True)
os.makedirs('embeddings/test', exist_ok=True)

In [ ]:
START_ID = 907
LIMIT = 100
writeEmbeddings(TRAIN_FILE, output_path='embeddings', start_id=START_ID, limit=LIMIT)

In [ ]:
from IPython.display import FileLink
#emb_path = "embeddings/train_id"+str(START_ID)+"_to"+str(START_ID+LIMIT)
emb_path = "embeddings/train"
shutil.make_archive(emb_path, 'zip', emb_path)
FileLink(r'embeddings/train.zip')

In [ ]:
!ls embeddings/train/

In [ ]:
embe01 = torch.load('embeddings/train/emb_seq0000.pt')
print(len(embe01))

embe02 = torch.load('embeddings/train/emb_seq0293.pt')
print(len(embe02))

print(embe01[0][0])
print(embe02[0][0])

## Embeddings visualization
https://krishansubudhi.github.io/deeplearning/2020/08/27/bert-embeddings-visualization.html  
https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin  
https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

In [4]:
os.makedirs('embeddings', exist_ok=True)
gdown.download(EMB_TRAIN, EMB_TRAIN_FILE, quiet=False)
EMB_TRAIN_ZIP = EMB_TRAIN_FILE + 'train.zip'
!unzip -q $EMB_TRAIN_ZIP 
!mv *.pt ./embeddings/train

In [5]:
!ls embeddings/train

In [11]:
import glob
import torch

def load_id_labels(path):
  labels = []
  with open(path, 'r') as input:
    line = input.readline()
    while line:
      line = input.readline()
      splitted_line = line.split('\t')
      if len(splitted_line) < 2:
        break
      seq_id = splitted_line[0]
      label = int(splitted_line[1])
      labels.append(label)

  return labels

def getEmbeddingDataset(path):
  embeddings = {}
  emb_array = []
  for filename in sorted(glob.glob(path + '/*.pt')):
    seq_id = int(filename.split('.pt')[0].split('seq')[1])
    if seq_id == len(labels): # erase the last one since it's empty
      break
    label = labels[seq_id]
    #print('SeqID:' + str(seq_id) + ' -> label = ' + str(label))
    tensors = torch.load(filename, map_location=torch.device('cpu'))
    embeddings[seq_id] = {'label': label, 
                          'tensors': tensors
                          }
    emb_array.append(tensors)
  return embeddings, emb_array
  # print('seq[' + str(seq_id) + '] label = ' + str(label))s(TRAIN_ID_LABELS_FILE)
#print(len(train_labels))

In [12]:
# downloading dataset id-label
gdown.download(TRAIN_ID_LABELS, TRAIN_ID_LABELS_FILE, quiet=False)

In [13]:
labels = load_id_labels(TRAIN_ID_LABELS_FILE)
n_mening = sum(labels)
print('Total train_ids: ' + str(len(labels)))
print('Carrier/Disease: ' + str(len(labels)-n_mening) + '/' + str(n_mening))

embeddings, emb_array = getEmbeddingDataset('embeddings/train')

In [15]:
labels[707]

In [ ]:
print(len(embeddings[0]['tensors']))

In [ ]:
# train_X = []
# train_y = []

# for key in embeddings:
#   label = embeddings[key]['label']
#   for emb in embeddings[key]['tensors']:
#     emb = embeddings[key]['tensors'][emb]
#     train_X.append(emb.tolist())
#     train_y.append(label)

# import pandas as pd
# df = pd.DataFrame(list(zip(train_X, train_y)),
#                columns =['X', 'y'])

In [19]:
embeddings = []
emb_labels = []
emb_seq_ids = []
for emb_file in sorted(glob.glob('embeddings/train/*.pt')):
    # embeddings/train/emb_seq0041.pt
    seq_id = int(emb_file.split('emb_seq')[-1].split('.pt')[0])
    seq_emb = torch.load(emb_file)
    for j in seq_emb:
        embeddings.append(seq_emb[j].cpu().numpy())
        emb_labels.append(labels[seq_id])
        emb_seq_ids.append(seq_id)

In [20]:
len(emb_labels)

In [21]:
pca = PCA(n_components=3)
pca_values = pca.fit_transform(embeddings)
pca_values.shape

In [22]:
import pandas as pd

column_values = ['X', 'Y', 'Z']
df = pd.DataFrame(data = pca_values, columns=column_values)
df['seq_id'] = emb_seq_ids
df['label'] = emb_labels
df

In [44]:
df[df['seq_id']==715]

In [23]:
y = df[df.columns[-1:]]
X = df[df.columns[:-2]]

X = X.values
y = y.values

print('X: ({},{})'.format(X.shape[0], X.shape[1]))
print('y: ({},{})'.format(y.shape[0], y.shape[1]))

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# scaler = StandardScaler()
# scaler = scaler.fit(X)

tr_X, ts_X, tr_y, ts_y = train_test_split(X, y, test_size=0.2, random_state=1)
tr_y = tr_y.ravel()

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

rfc = RandomForestClassifier()
rfc.fit(tr_X,tr_y)

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

# predictions
rfc_predict = rfc.predict(ts_X)
rfc_cv_score = cross_val_score(rfc, tr_X, tr_y, cv=4, scoring='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(ts_y, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(ts_y, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

In [27]:
values = pd.DataFrame(df['seq_id'].value_counts()).sort_values(by=['seq_id'], ascending=False)
values

In [33]:
import plotly.express as px

def project_embeddings_3d(df, x, y, z, color):
    df_copy = df.copy()
    df_copy["label"] = df["label"].astype(str)
    fig = px.scatter_3d(df_copy, x=x, y=y, z=z, color=color)
    fig.update_traces(marker=dict(size=3),
                  selector=dict(mode='markers'))
    fig.show()

In [34]:
def plot2DSeq(id, df=df, axes=['X', 'Y']):
    seq = df[df['seq_id'] == id]
    px.scatter(seq, x=axes[0], y=axes[1], color='label')
    
def plotSeq(id, df=df):
    seq = df[df['seq_id'] == id]
    project_embeddings_3d(seq, x='X', y='Y', z='Z', color='label')
    
def plotCompare(ids, df=df):
    seq = df[df["seq_id"].isin(ids)]
    project_embeddings_3d(seq, x='X', y='Y', z='Z', color='label')

In [40]:
labels[715]

In [38]:
#plotSeq(293)
#plotSeq(715)

numbers = [293, 715]
#numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 707, 708, 709, 710, 711, 712, 713, 714]
plotCompare(numbers)

In [ ]:
numbers = [0, 293, 715]
seq = df[df["seq_id"].isin(numbers)]
px.scatter(seq, x='X', y='Y', color='label')

## Classification on Embeddings (Random Forest)

In [ ]:
# import tensorflow.keras.backend as K
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier

# from sklearn.model_selection import KFold


# forest = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = forest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
# rf_random.fit(train_X, train_y)
